In [46]:
pip install pandas numpy seaborn matplotlib pymorphy2 scikit-learn tqdm

In [47]:
import pandas as pd

df = pd.read_csv("разметка комментариев 2.csv")

df.shape


(2373, 8)

In [ ]:
# Удаление дубликатов 
df = df.drop_duplicates(subset='comment')

df.shape  


(1812, 8)

In [49]:
df.to_csv("разметка_без_дубликатов.csv", index=False)
print("Файл сохранен как 'разметка_без_дубликатов.csv'")


Файл сохранен как 'разметка_без_дубликатов.csv'


In [50]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

def lemmatize_text(text):
    words = str(text).lower().split()
    lemmas = [morph.parse(word)[0].normal_form for word in words]
    return lemmas

# Ключевые слова по категориям
keyword_map = {
    "Нравится скорость отработки заявок": ["быстро", "оперативно", "скорость"],
    "Нравится качество выполнения заявки": ["качественно", "отлично", "хорошо"],
    "Нравится качество работы сотрудников": ["вежливо", "спасибо", "благодарю", "профессионально", "мастер", "специалист", "сотрудник",],
    "Понравилось выполнение заявки": ["выполнить", "сделать", "хорошо", "спасибо", "благодарю", ],
    "Вопрос решен": ["решить", "устранить", "помочь" "закрыт", "выполнена",]
}

def classify_comment(comment):
    lemmas = lemmatize_text(comment)
    result = {}
    for category, keywords in keyword_map.items():
        result[category] = int(any(word in lemmas for word in keywords))
    return result


In [ ]:
import pandas as pd
import pymorphy2
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm

tqdm.pandas()
morph = pymorphy2.MorphAnalyzer()

# Лемматизация
def lemmatize_text(text):
    words = str(text).lower().split()
    lemmas = [morph.parse(word)[0].normal_form for word in words]
    return " ".join(lemmas)

# Загрузка размеченного CSV
df = pd.read_csv("readyrazmet.csv")  
df.dropna(subset=["comment"], inplace=True)

# Лемматизация текста
df["lemmatized"] = df["comment"].progress_apply(lemmatize_text)

# Векторизация TF-IDF
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
X = vectorizer.fit_transform(df["lemmatized"])

# Целевые переменные
target_columns = [col for col in df.columns if col not in ["comment", "lemmatized", "id", "rating"]]

y = df[target_columns]

print(f"Матрица признаков: {X.shape}")
print(f"Целевые переменные: {y.shape}")


100%|██████████| 1812/1812 [00:02<00:00, 604.35it/s]


Матрица признаков: (1812, 5000)
Целевые переменные: (1812, 6)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import hamming_loss, accuracy_score

# Разделение на обучающую и тестовую выборку
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Инициализация модели
base_model = RandomForestClassifier(n_estimators=100, random_state=42)
model = MultiOutputClassifier(base_model)



# Обучение
model.fit(X_train, y_train)



# Предсказание
y_pred = model.predict(X_test)



# Оценка модели
print("✅ Отчет по качеству классификации:")

for i, column in enumerate(y.columns):
    print(f"\nКатегория: {column}")
    print(classification_report(y_test.iloc[:, i], y_pred[:, i]))

✅ Отчет по качеству классификации:

Категория: Нравится скорость отработки заявок
              precision    recall  f1-score   support

           0       0.92      0.94      0.93       287
           1       0.76      0.67      0.71        76

    accuracy                           0.89       363
   macro avg       0.84      0.81      0.82       363
weighted avg       0.88      0.89      0.88       363


Категория: Нравится качество выполнения заявки
              precision    recall  f1-score   support

           0       0.97      0.97      0.97       345
           1       0.40      0.33      0.36        18

    accuracy                           0.94       363
   macro avg       0.68      0.65      0.67       363
weighted avg       0.94      0.94      0.94       363


Категория: Нравится качество работы сотрудников
              precision    recall  f1-score   support

           0       0.87      0.91      0.89       234
           1       0.83      0.75      0.79       129

   